<a href="https://colab.research.google.com/github/sasayagogo/SECOM_AI/blob/main/Outliers_Detection_SECOM_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
import os
os.chdir("/content/drive/MyDrive/colab/PHM/221219_SECOM_AI/") #更改路徑
os.getcwd() #查看當前路徑

'/content/drive/MyDrive/colab/PHM/221219_SECOM_AI'

In [ ]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns
#test

In [ ]:
# reading the data
data = pd.read_csv('dataset/uci-secom.csv')

# getting the shape of the data
print(data.shape)

(1567, 592)


In [ ]:
data.head()

,Time,0,1,2,3,4,5,6,7,8,...,581,582,583,584,585,586,587,588,589,Pass/Fail
0,2008-07-19 11:55:00,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,...,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,-1
1,2008-07-19 12:32:00,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,...,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1
2,2008-07-19 13:17:00,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,...,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1
3,2008-07-19 14:43:00,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,...,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432,-1
4,2008-07-19 15:22:00,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,...,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432,-1


In [ ]:
# droping the first column of the data

data = data.drop(['Time'], axis = 1)

In [ ]:
# filling missing values

data.fillna(0, inplace = True)

data.isnull().sum().sum()

0

In [ ]:
# again creating x and y from the dataset

x = data.iloc[:, :-1]
y = data.iloc[:, -1]

# getting the shapes
print("Shape of x:", x.shape)
print("Shape of y:", y.shape)

Shape of x: (1567, 590)
Shape of y: (1567,)


In [ ]:
# splittng the dataset

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

# print the shapes
print("Shape of x_train :", x_train.shape)
print("Shape of x_test :", x_test.shape)
print("Shape of y_train :", y_train.shape)
print("Shape of y_test :", y_test.shape)

Shape of x_train : (1253, 590)
Shape of x_test : (314, 590)
Shape of y_train : (1253,)
Shape of y_test : (314,)


In [ ]:
# defining outlier fraction

Fraud = data[data['Pass/Fail']==1] #fail
Valid = data[data['Pass/Fail']==-1] #normal

outlier_fraction = len(Fraud)/float(len(Valid)) #fail的比例計算
print("Outlier Fraction :", outlier_fraction)

Outlier Fraction : 0.0710868079289132


In [ ]:
# importing the libraries
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM

classifiers = {
    "Isolation Forest":IsolationForest(n_estimators=100, 
                      max_samples=len(x_train), 
                      contamination=outlier_fraction,
                      random_state=0, 
                      verbose=0),
               
    "Local Outlier Factor":LocalOutlierFactor(n_neighbors=20, 
                          algorithm='auto', 
                          leaf_size=30, 
                          metric='minkowski',
                          p=2, 
                          metric_params=None, 
                          contamination=outlier_fraction),
               
    "Support Vector Machine":OneClassSVM(kernel='rbf', 
                        degree=3, 
                        gamma=0.1,
                        nu=0.05, 
                        max_iter=-1)
        #"Support Vector Machine":OneClassSVM(kernel='rbf', degree=3, gamma=0.1,nu=0.05, 
                        #max_iter=-1, random_state=1)
   
}


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

n_outliers = len(Fraud)
for i, (clf_name,clf) in enumerate(classifiers.items()):
    #Fit the data and tag outliers
    if clf_name == "Local Outlier Factor":
        y_pred = clf.fit_predict(x_test)
        scores_prediction = clf.negative_outlier_factor_
    elif clf_name == "Support Vector Machine":
        clf.fit(x_train)
        y_pred = clf.predict(x_test)
    else:    
        clf.fit(x_train)
        scores_prediction = clf.decision_function(x_test)
        y_pred = clf.predict(x_test)
    #Reshape the prediction values to 0 for Valid transactions , 1 for Fraud transactions
    y_pred[y_pred == 1] = 0
    y_pred[y_pred == -1] = 1
    n_errors = (y_pred != y_test).sum()
    # Run Classification Metrics
    print("{}: {}".format(clf_name,n_errors))
    print("Accuracy Score :")
    print(accuracy_score(y_test,y_pred))
    print("Classification Report :")
    print(classification_report(y_test,y_pred))


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


Isolation Forest: 312
Accuracy Score :
0.006369426751592357
Classification Report :
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       301
           0       0.00      0.00      0.00         0
           1       0.11      0.15      0.12        13

    accuracy                           0.01       314
   macro avg       0.04      0.05      0.04       314
weighted avg       0.00      0.01      0.01       314

Local Outlier Factor: 312
Accuracy Score :
0.006369426751592357
Classification Report :
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       301
           0       0.00      0.00      0.00         0
           1       0.09      0.15      0.11        13

    accuracy                           0.01       314
   macro avg       0.03      0.05      0.04       314
weighted avg       0.00      0.01      0.00       314



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

Support Vector Machine: 301
Accuracy Score :
0.041401273885350316
Classification Report :
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       301
           1       0.04      1.00      0.08        13

    accuracy                           0.04       314
   macro avg       0.02      0.50      0.04       314
weighted avg       0.00      0.04      0.00       314



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
